In [ ]:
# установка катбуста и активация нужных параметров
# !pip install catboost
# !pip install ipywidgets
# !jupyter nbextension enable --py widgetsnbextension

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv("./data/mushrooms_small.tsv", sep='\t')
df

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
# сколько категорий в каждом столбце
df.nunique()

In [ ]:
df.isna().sum()

In [ ]:
X, y = df.drop(columns='class'), df['class'] # отделим лейблы

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=3+2+2024)

# Обучим как-то простую модель

In [ ]:
y.value_counts()

In [ ]:
X_encoded = pd.get_dummies(X, drop_first=True, dtype=int)
y_encoded = y.map({'p':1, 'e':0})

In [ ]:
X_encoded.columns

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_encoded, test_size=0.25, random_state=3+2+2024)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
models = dict(
    LogReg = LogisticRegression(),
    KNN = KNeighborsClassifier(),
    SVM = SVC(),
    Tree = DecisionTreeClassifier(),
    RandomForest = RandomForestClassifier()
)

In [ ]:
prefit = {}
for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    prefit[model_name] = dict(model=model, score=f1_score(y_test, y_pred))
prefit

In [ ]:
imp = pd.DataFrame({'cols': X_train.columns, 'coef':prefit['LogReg']['model'].coef_[0]})
imp['abs'] = imp.coef.abs()
imp = imp.sort_values(by='abs', ascending=False)
imp.head(10)

# Catboost

In [ ]:
import catboost
catboost.__version__

In [ ]:
from catboost import CatBoostClassifier, Pool

In [ ]:
# создаём простую модель
# https://catboost.ai/en/docs/concepts/python-reference_catboostclassifier

model = CatBoostClassifier(iterations=100,
                           depth=3,
                           learning_rate=0.5,
                   # если класс -- лейбл, используем LogLoss. Если класс -- вероятность, используем кросс-энтропию
                           loss_function='Logloss', 
                           verbose=True)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.25, random_state=3+2+2024)

In [ ]:
# подготовим данные в нужном формате
data_train = Pool(data=X_train, label=y_train, cat_features=X_train.columns.values)
data_test = Pool(data=X_test, label=y_test, cat_features=X_test.columns.values)

In [ ]:
# учим модель
model.fit(data_train)

In [ ]:
# реже выводим информацию
model.fit(data_train, 
         metric_period = 10)

In [ ]:
# интерактивный монитор
model.fit(data_train, 
          plot=True, 
          silent=True)

In [ ]:
print("Model is fitted: {}".format(model.is_fitted()))
print(model.get_params())
print("Best score: {}".format(model.best_score_))

## Добавим в монитор другие метрики

In [ ]:
# учим модель
model = CatBoostClassifier(iterations=100,
                                   depth=3,
                                   learning_rate=0.5,
                                   loss_function='Logloss', 
                                   custom_loss=['AUC', 'F1', 'Accuracy'])

model.fit(data_train, 
          plot=True, 
          silent=True)
         

## Добавим валидацию

In [ ]:
# учим модель
model = CatBoostClassifier(iterations=100,
                                   depth=3,
                                   learning_rate=0.5,
                                   loss_function='Logloss', 
                                   custom_loss=['AUC', 'F1', 'Accuracy'])

model.fit(data_train, 
          # plot=True, 
          # silent=True,
          eval_set=data_test)
         

In [ ]:
model.fit(data_train, eval_set=data_test, metric_period=10)

In [ ]:
# увеличим число деревьев в два раза, до 200
model = CatBoostClassifier(iterations=200,
                                   depth=3,
                                   learning_rate=0.5,
                                   loss_function='Logloss', 
                                   eval_metric='F1')

model.fit(data_train, 
          # plot=True, 
          # silent=True,
          eval_set=data_test,
          metric_period = 25)
         

In [ ]:
# Оптимизируется logloss, но наши целевые метрики не растут с одной из самых ранних итераций

In [ ]:
# поменяем оптимизируемую метрику на AUC
model = CatBoostClassifier(iterations=100,
                                   depth=3,
                                   learning_rate=0.5,
                                   loss_function='Logloss', 
                                   eval_metric='AUC')

# model.fit(data_train, 
#           plot=True, 
#           silent=True,
#           eval_set=data_test)
         

In [ ]:
# поменяем оптимизируемуб метрику на AUC
model = CatBoostClassifier(iterations=200,
                                   depth=3,
                                   learning_rate=0.5,
                                   loss_function='Logloss', 
                                   eval_metric='F1')

# добавим остановку обучения, если целевая метрика НЕ МЕНЯЕТСЯ в течение некоторого числа итераций
model.fit(data_train, 
          eval_set=data_test,
          early_stopping_rounds=20)
         

# Сравним модели

In [ ]:
model1 = CatBoostClassifier(iterations=100,
                            depth=3,
                            learning_rate=0.5,
                            loss_function='Logloss', 
                            eval_metric='F1',
                            train_dir='lr0.5')
model2 = CatBoostClassifier(iterations=100,
                            depth=3,
                            learning_rate=0.01,
                            loss_function='Logloss', 
                            eval_metric='F1',
                            train_dir='lr1')


model1.fit(data_train, 
           eval_set=data_test,
           silent=True,)
model2.fit(data_train, 
           eval_set=data_test,
           silent=True,)

In [ ]:
from catboost import MetricVisualizer
MetricVisualizer(['lr0.5', 'lr1']).start()

# Кросс-валидация

In [ ]:
from catboost import cv

params = {'loss_function': 'Logloss',
          'iterations': 100,
          'eval_metric': 'F1',
          'learning_rate':0.2}

cv_data = cv(
    params = params,
    pool = data_train,
    fold_count = 5,
    shuffle = True,
    partition_random_seed = 42,
    verbose=False,
   # plot = True,
   # stratified=True # пропорции в каждом фолде такие же, как и во всех данных
)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

param_grid = {
    "learning_rate": [0.0001, 0.01, 0.25, 0.7]
}

model = CatBoostClassifier(
    iterations = 50,
    metric_period=50,
    verbose=True,
    eval_metric='F1',
    cat_features=X_train.columns.values
)

scorer = make_scorer(f1_score, greater_is_better=True)

grid_search = GridSearchCV(model, param_grid=param_grid, cv=5, verbose=True, n_jobs=1, scoring=scorer)
results = grid_search.fit(X_train, y_train)

In [ ]:
results.best_estimator_

In [ ]:
results.best_estimator_.get_params()

# Предскажем и оценим

In [ ]:
model = CatBoostClassifier(
    eval_metric='F1',
    iterations=200,
    learning_rate=0.5,
    early_stopping_rounds=20
)

model.fit(
    data_test,
    eval_set=data_test,
#     silent=True,
#     plot=True
)

In [ ]:
model.get_params()

In [ ]:
model.predict(X_test)

In [ ]:
model.predict_proba(X_test) # вероятности (получили путём применения сигмоиды к полученным значениями в листьях)

In [ ]:
# сырые значения функции предсказания
model.predict(X_test, prediction_type='RawFormulaVal')

$$ S(x) = \frac{1}{1 + e^{-x}}$$

In [ ]:
sigmoid = lambda x : 1 / (1 + np.exp(-x)) # написали сами сигмоиду

raw = model.predict(X_test, prediction_type='RawFormulaVal')

probabilities = sigmoid(raw)

In [ ]:
(model.predict_proba(X_test)[:, 1] != probabilities).sum() # вероятности принадлежать целевому классу совпали везде

 # Feature importance

In [ ]:
model.get_feature_importance(prettified=True)

In [ ]:
model.get_feature_importance(prettified=True).Importances.sum()

# SHAP

https://medium.com/dataman-in-ai/explain-your-model-with-the-shap-values-bc36aac4de3d
<img src="https://raw.githubusercontent.com/slundberg/shap/master/docs/artwork/mnist_image_plot.png" />

In [ ]:
# !pip install shap

In [ ]:
shap_values = model.get_feature_importance(data_test, 'ShapValues')
shap_values

In [ ]:
X_test.shape

In [ ]:
shap_values.shape

In [ ]:
expected_value = shap_values[1, -1]
expected_value

In [ ]:
shap_values = shap_values[:, :-1]

In [ ]:
shap_values[1, :].shape

In [ ]:
y_test.values[1]

In [ ]:
import shap

shap.initjs()
shap.force_plot(expected_value, shap_values[1, :], X_test.iloc[1, :])

In [ ]:
y_test.values[5]

In [ ]:
shap.initjs() # пример, где признаки вносят "отрицательный" вклад
shap.force_plot(expected_value, shap_values[5, :], X_test.iloc[5, :])

In [ ]:
shap_values = model.get_feature_importance(data_test, 'ShapValues')
shap_values

In [ ]:
shap_values = model.get_feature_importance(data_test, 'ShapValues')
shap_values = shap_values[:, :-1] # последний столбец -- среднее значение "функции" (оно одинаково для всех. это свободный член)
shap.summary_plot(shap_values, X_test)

# окраски нет, потому что данные категориальные, а их кодирование в числа происходит внутри 

In [ ]:
X_test.shape

# Практика

In [ ]:
from sklearn.datasets import load_digits # MNIST

In [ ]:
X, y = load_digits(n_class=10, return_X_y=True, as_frame=False)

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
plt.figure(figsize=(16, 6))
for i in range(10):
    plt.subplot(2, 5, i + 1)
    plt.imshow(X[i,:].reshape([8,8]));

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=15062023, test_size=0.25)

# Многоклассовая логистическая регрессия

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_sc = scaler.fit_transform(X_train)
X_test_sc = scaler.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
model = LogisticRegression()

model.fit(X_train, y_train)

accuracy_score(y_test, model.predict(X_test))

In [ ]:
model = LogisticRegression()

model.fit(X_train_sc, y_train)

accuracy_score(y_test, model.predict(X_test_sc))

# SVM

# Метод K-ближайших соседей

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
params = {'n_neighbors': [2, 3, 4, 5, 7, 10, 15]}

model = GridSearchCV(KNeighborsClassifier(), params, verbose=False)

model.fit(X_train, y_train)

print(model.best_params_)
accuracy_score(y_test, model.best_estimator_.predict(X_test))

In [ ]:
params = {'n_neighbors': [2, 3, 4, 5, 7, 10, 15]}

model = GridSearchCV(KNeighborsClassifier(), params, verbose=False)

model.fit(X_train_sc, y_train)

print(model.best_params_)
accuracy_score(y_test, model.best_estimator_.predict(X_test_sc))

# Дерево решений

# Случайный лес

# Градиентный бустинг решающих деревьев

In [ ]:
from catboost import CatBoostClassifier, Pool

In [ ]:
data_train = Pool(data=X_train, label=y_train)
data_test = Pool(data=X_test, label=y_test)

In [ ]:
model = CatBoostClassifier(iterations=100,
                           early_stopping_rounds=10,
                           eval_metric='AUC')

In [ ]:
model.fit(data_train, eval_set=data_test)

In [ ]:
accuracy_score(y_test, model.predict(data_test))

In [ ]:
shap_values.shape

In [ ]:
shap_values = model.get_feature_importance(data_test, 'ShapValues')

In [ ]:
shap_values.shape

In [ ]:
shap_values = shap_values[:, :, :-1] # последний столбец -- среднее значение "функции" (оно одинаково для всех. это свободный член)

In [ ]:
shap_values.shape

In [ ]:

shap.summary_plot(shap_values[:, 3, :], X_test)

# окраски нет, потому что данные категориальные, а их кодирование в числа происходит внутри 